# Advanced Prophet Usage
You can scan through [prophet docs](https://facebook.github.io/prophet/docs/quick_start.html) and find many options how to tweak your model. 
Some of that functionality is moved to initialization stage to be compatible with Sklearn API. We will showcase the parts that were moved to initialization, but you can also look for other model parameters that could help fine-tuning your model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
from hcrystalball.utils import generate_tsdata
X, y = generate_tsdata(n_dates=365*2)

In [ ]:
from hcrystalball.wrappers import ProphetWrapper

In [ ]:
ProphetWrapper?

## Advanced Holidays
For holidays, we are able to define instead of single boolean attribute distribution around given day. We define `lower_window`, `upper_window` and `prior_scales`

In [ ]:
extra_holidays = {
    'Christmas Day':{'lower_window': -2, 'upper_window':2, 'prior_scale': 20},
#     'Good Friday':{'lower_window': -1, 'upper_window':1, 'prior_scale': 30}
}

## Unusual Seasonalities

In [ ]:
extra_seasonalities = [
    {
        'name':'bi-weekly',
        'period': 14.,
        'fourier_order': 5,
        'prior_scale': 10.0,
        'mode': None
    },
    {
        'name':'bi-yearly',
        'period': 365*2.,
        'fourier_order': 5, 
        'prior_scale': 5.0,
        'mode': None
    },
]

## Exogenous Variables

In [ ]:
from sklearn.pipeline import Pipeline
from hcrystalball.feature_extraction import HolidayTransformer

In [ ]:
extra_regressors = ['trend_line']
X['trend_line'] = np.arange(len(X))

In [ ]:
prophet = ProphetWrapper(
    name='prophet',
)
prophet_extra = ProphetWrapper(
    extra_holidays=extra_holidays,
    extra_seasonalities=extra_seasonalities,
    extra_regressors=extra_regressors,
    name='prophet_extra',
)

In [ ]:
pipeline = Pipeline([
    ('holidays_de', HolidayTransformer(country_code = 'DE')),
    ('model', prophet)
]) 
pipeline_extra = Pipeline([
    ('holidays_de', HolidayTransformer(country_code = 'DE')),
    ('model', prophet_extra)
]) 

In [ ]:
prds = (pipeline.fit(X[:-10], y[:-10])
         .predict(X[-10:])
         .merge(y, left_index=True, right_index=True, how='outer')
         .tail(50))

prds.plot(title=f"MAE:{(prds['target']-prds['prophet']).abs().mean().round(3)}");

In [ ]:
prds_extra = (pipeline_extra.fit(X[:-10], y[:-10])
         .predict(X[-10:])
         .merge(y, left_index=True, right_index=True, how='outer')
         .tail(50))

prds_extra.plot(title=f"MAE:{(prds_extra['target']-prds_extra['prophet_extra']).abs().mean().round(3)}");

Compared to non-tweaked model, we are able to better catch the series dynamics, but don't win in MAE against roughly average predictions

In [ ]:
prds = (ProphetWrapper().fit(X[:-10], y[:-10])
                 .predict(X[-10:])
                 .merge(y, left_index=True, right_index=True, how='outer')
                 .tail(50)
)
prds.plot(title=f"MAE:{(prds['target']-prds['prophet']).abs().mean().round(3)}");

## Full Prophet Output
If you need, you can also pass `full_prophet_output` and get rich predict output

In [ ]:
(ProphetWrapper(full_prophet_output=True, conf_int=True)
     .fit(X[:-10], y[:-10])
     .predict(X[-10:])
)